# 📊 TelecomX - Análise de Evasão de Clientes
## Challenge Data Science | Alura + Oracle Next Education (ONE)

---

**👤 Autor:** Fábio Andrade  
**📅 Data:** 28 de Dezembro de 2024  
**🔗 Repositório:** [GitHub - telecomx-churn-analysis](https://github.com/thedrads/telecomx-churn-analysis)  
**🎯 Objetivo:** Análise completa de Churn usando processo ETL + EDA preparatória para Machine Learning

---

### 📌 Sobre este Projeto

Este notebook documenta a análise de dados de evasão de clientes (churn) da empresa **TelecomX**, desenvolvido como parte do Challenge de Data Science da Alura.

**Etapas do projeto:**
1. 📥 **Extract** - Extração de dados via API REST
2. 🔧 **Transform** - Limpeza e transformação dos dados
3. 💾 **Load** - Carga dos dados tratados
4. 📊 **EDA** - Análise Exploratória de Dados
5. 💡 **Insights** - Recomendações estratégicas

---

### 🎯 Diferenciais deste Projeto

- ✅ **Colunas traduzidas para português** (facilita compreensão e ML futuro)
- ✅ **ETL rigoroso** com análise de qualidade profunda
- ✅ **Documentação de decisões** (ex: tratamento de valores vazios)
- ✅ **Preparado para Machine Learning** (tipos corretos, sem missing values)
- ✅ **Seguindo boas práticas** de Data Science internacional

---

## 🏢 1. CONTEXTO DO NEGÓCIO

### 📉 O Problema: Evasão de Clientes (Churn)

A **TelecomX** é uma empresa de telecomunicações que enfrenta um desafio crítico: **alto índice de cancelamento de contratos**.

#### 🎯 O que é Churn?

**Churn** (ou evasão) refere-se ao cancelamento de serviços. Na telecomunicações:
- ❌ Cliente cancela contrato
- ❌ Para de pagar pelos serviços
- ❌ Migra para concorrência

#### 💰 Impacto no Negócio

> **Estatística chave:** Reter um cliente custa até 7 vezes menos que adquirir um novo.

---

### 🎯 Objetivos do Projeto

1. **Extrair** dados de clientes do sistema (API REST)
2. **Limpar e transformar** dados para análise
3. **Identificar padrões** que diferenciam clientes que cancelam vs. permanecem
4. **Gerar insights acionáveis** para retenção
5. **Preparar dataset** para futuros modelos de Machine Learning

#### 📊 Perguntas de Negócio

- ❓ Qual o perfil demográfico dos clientes que mais cancelam?
- ❓ Quais serviços estão associados a maior/menor churn?
- ❓ Tipo de contrato influencia na retenção?
- ❓ Qual o impacto financeiro do churn atual?

---

## 📖 2. DICIONÁRIO DE DADOS

### 📋 Estrutura do Dataset

**Total:** 21 variáveis organizadas em 5 categorias

---

#### 🆔 IDENTIFICAÇÃO
- `customerID` → `id_cliente`: Identificador único

#### 🎯 VARIÁVEL ALVO (Target para ML)
- `Churn` → `cancelou`: Cliente cancelou? (Yes/No)

#### 👥 VARIÁVEIS DEMOGRÁFICAS
- `gender` → `genero`: Masculino/Feminino
- `SeniorCitizen` → `idoso`: 1=Sim, 0=Não (65+ anos)
- `Partner` → `tem_parceiro`: Yes/No
- `Dependents` → `tem_dependentes`: Yes/No

#### 📞 SERVIÇOS CONTRATADOS

**Telefone:**
- `PhoneService` → `servico_telefone`: Yes/No
- `MultipleLines` → `linhas_multiplas`: Yes/No/No phone service

**Internet:**
- `InternetService` → `tipo_internet`: DSL/Fiber optic/No
- `OnlineSecurity` → `seguranca_online`: Yes/No/No internet service
- `OnlineBackup` → `backup_online`: Yes/No/No internet service
- `DeviceProtection` → `protecao_dispositivo`: Yes/No/No internet service
- `TechSupport` → `suporte_tecnico`: Yes/No/No internet service
- `StreamingTV` → `streaming_tv`: Yes/No/No internet service
- `StreamingMovies` → `streaming_filmes`: Yes/No/No internet service

#### 💰 INFORMAÇÕES FINANCEIRAS
- `tenure` → `meses_cliente`: Meses como cliente (0-72)
- `Contract` → `tipo_contrato`: Month-to-month/One year/Two year
- `PaperlessBilling` → `fatura_digital`: Yes/No
- `PaymentMethod` → `metodo_pagamento`: 4 opções
- `MonthlyCharges` → `cobranca_mensal`: USD (18-120)
- `TotalCharges` → `cobranca_total`: USD acumulado

---

### 📌 Nota sobre Tradução

Seguindo recomendação dos instrutores da Alura, **todas as colunas foram traduzidas para português** para:
- ✅ Facilitar compreensão e manutenção do código
- ✅ Tornar apresentações mais acessíveis
- ✅ Preparar para modelagem em português (se necessário)

---

In [1]:
# ====================================================================
# 📦 INSTALAÇÃO DE DEPENDÊNCIAS
# ====================================================================

print("🔄 Atualizando pip...")
!pip install -q --upgrade pip

print("\n📦 Instalando Plotly para gráficos interativos...")
!pip install -q plotly==5.18.0

print("\n" + "="*60)
print("✅ INSTALAÇÃO CONCLUÍDA!")
print("="*60)

🔄 Atualizando pip...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 16.1 MB/s eta 0:00:00

📦 Instalando Plotly para gráficos interativos...

✅ INSTALAÇÃO CONCLUÍDA!


In [2]:
# ====================================================================
# 📚 IMPORTAÇÃO DE BIBLIOTECAS
# ====================================================================

# Manipulação de dados
import pandas as pd
import numpy as np

# Requisições HTTP
import requests
import json

# Visualização
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

# Utilitários
from datetime import datetime
import warnings
import os

# ====================================================================
# 🎨 CONFIGURAÇÕES GLOBAIS
# ====================================================================

warnings.filterwarnings('ignore')

# Pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.2f}'.format)

# Seaborn
sns.set_style("whitegrid")
sns.set_palette("Set2")

# Matplotlib
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

# ====================================================================
# 🎨 PALETA DE CORES DO PROJETO
# ====================================================================

CORES = {
    'primary': '#2E86AB',
    'secondary': '#A23B72',
    'accent': '#F18F01',
    'success': '#06A77D',
    'danger': '#D81159',
    'warning': '#F18F01',
    'info': '#2E86AB',
    'neutral': '#6C757D',
    'churn_yes': '#E63946',
    'churn_no': '#06FFA5',
    'palette': ['#2E86AB', '#A23B72', '#F18F01', '#06A77D',
                '#D81159', '#6C757D', '#E63946', '#06FFA5']
}

print("="*60)
print("✅ BIBLIOTECAS IMPORTADAS COM SUCESSO!")
print("="*60)
print(f"\n📊 Versões:")
print(f"   • Pandas: {pd.__version__}")
print(f"   • NumPy: {np.__version__}")
print(f"   • Matplotlib: {plt.matplotlib.__version__}")
print(f"   • Seaborn: {sns.__version__}")
print("\n" + "="*60)
print("🎯 AMBIENTE CONFIGURADO!")
print("="*60)

✅ BIBLIOTECAS IMPORTADAS COM SUCESSO!

📊 Versões:
   • Pandas: 2.2.2
   • NumPy: 2.0.2
   • Matplotlib: 3.10.0
   • Seaborn: 0.13.2

🎯 AMBIENTE CONFIGURADO!


---

## 📥 3. EXTRAÇÃO DE DADOS (Extract)

### 🎯 Objetivo

Conectar-se à API REST da TelecomX e extrair dados brutos de clientes em formato JSON.

**Fonte:** `https://raw.githubusercontent.com/ingridcristh/challenge2-data-science/main/TelecomX_Data.json`

### 🔧 Técnicas

- `requests.get()` - Requisição HTTP
- `pd.json_normalize()` - Normalização de JSON aninhado
- Tratamento de erros robusto

---

In [3]:
# ====================================================================
# 🌐 FUNÇÃO DE EXTRAÇÃO DE DADOS DA API
# ====================================================================

def extrair_dados_api(url, timeout=30):
    """
    Extrai dados de uma API REST e retorna como DataFrame.

    Parâmetros:
    -----------
    url : str
        URL da API REST
    timeout : int
        Tempo máximo de espera (segundos)

    Retorna:
    --------
    pd.DataFrame ou None
    """

    try:
        print(f"🌐 Conectando à API...")
        print(f"   URL: {url}")

        response = requests.get(url, timeout=timeout)

        if response.status_code == 200:
            print(f"   ✅ Status: {response.status_code} (OK)")

            dados_json = response.json()

            if isinstance(dados_json, list):
                print(f"   ✅ Tipo: Lista com {len(dados_json):,} registros")
            else:
                print(f"   ⚠️ Tipo inesperado: {type(dados_json)}")
                return None

            print(f"🔄 Convertendo para DataFrame...")
            df = pd.DataFrame(dados_json)

            print(f"   ✅ DataFrame criado: {df.shape[0]:,} linhas × {df.shape[1]} colunas")

            return df

        else:
            print(f"   ❌ ERRO HTTP: Status {response.status_code}")
            return None

    except requests.exceptions.Timeout:
        print(f"❌ ERRO: Timeout após {timeout} segundos")
        return None

    except requests.exceptions.RequestException as e:
        print(f"❌ ERRO de Requisição: {str(e)}")
        return None

    except json.JSONDecodeError as e:
        print(f"❌ ERRO ao parsear JSON: {str(e)}")
        return None

    except Exception as e:
        print(f"❌ ERRO inesperado: {str(e)}")
        return None

print("="*60)
print("✅ FUNÇÃO extrair_dados_api() DEFINIDA!")
print("="*60)

✅ FUNÇÃO extrair_dados_api() DEFINIDA!


In [4]:
# ====================================================================
# 🚀 EXECUTAR EXTRAÇÃO DE DADOS
# ====================================================================

print("="*60)
print("🚀 INICIANDO EXTRAÇÃO")
print("="*60)

API_URL = "https://raw.githubusercontent.com/ingridcristh/challenge2-data-science/main/TelecomX_Data.json"

df_raw = extrair_dados_api(API_URL)

print("\n" + "="*60)
if df_raw is not None:
    print("✅ EXTRAÇÃO BEM-SUCEDIDA!")
    print("="*60)

    print(f"\n📊 Dimensões: {df_raw.shape[0]:,} registros × {df_raw.shape[1]} colunas")
    print(f"\n📋 Colunas extraídas:")
    for i, col in enumerate(df_raw.columns, 1):
        print(f"   {i}. {col}")

    print(f"\n🔍 Preview (primeiras 3 linhas):")
    print("="*60)
    display(df_raw.head(3))

    print(f"\n📦 Informações técnicas:")
    print("="*60)
    df_raw.info()
else:
    print("❌ FALHA NA EXTRAÇÃO!")
    print("="*60)

🚀 INICIANDO EXTRAÇÃO
🌐 Conectando à API...
   URL: https://raw.githubusercontent.com/ingridcristh/challenge2-data-science/main/TelecomX_Data.json
   ✅ Status: 200 (OK)
   ✅ Tipo: Lista com 7,267 registros
🔄 Convertendo para DataFrame...
   ✅ DataFrame criado: 7,267 linhas × 6 colunas

✅ EXTRAÇÃO BEM-SUCEDIDA!

📊 Dimensões: 7,267 registros × 6 colunas

📋 Colunas extraídas:
   1. customerID
   2. Churn
   3. customer
   4. phone
   5. internet
   6. account

🔍 Preview (primeiras 3 linhas):


,customerID,Churn,customer,phone,internet,account
0,0002-ORFBO,No,"{'gender': 'Female', 'SeniorCitizen': 0, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'One year', 'PaperlessBilling': '..."
1,0003-MKNFE,No,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'Yes'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
2,0004-TLHLJ,Yes,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."



📦 Informações técnicas:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   customerID  7267 non-null   object
 1   Churn       7267 non-null   object
 2   customer    7267 non-null   object
 3   phone       7267 non-null   object
 4   internet    7267 non-null   object
 5   account     7267 non-null   object
dtypes: object(6)
memory usage: 340.8+ KB


In [5]:
# ====================================================================
# 🔧 NORMALIZAÇÃO DE JSON ANINHADO
# ====================================================================

print("="*60)
print("🔧 NORMALIZANDO ESTRUTURA JSON")
print("="*60)

if df_raw is not None:

    print(f"\n📊 ANTES da normalização: {df_raw.shape[1]} colunas")

    print("\n🔄 Aplicando pd.json_normalize()...")
    df_normalized = pd.json_normalize(df_raw.to_dict('records'))

    print(f"   ✅ Normalização concluída!")
    print(f"\n📊 DEPOIS da normalização: {df_normalized.shape[1]} colunas")

    print(f"\n📋 Colunas normalizadas (com prefixos):")
    for i, col in enumerate(df_normalized.columns, 1):
        print(f"   {i:2d}. {col}")

    print(f"\n🔍 Preview:")
    print("="*60)
    display(df_normalized.head())

    print("\n" + "="*60)
    print("✅ NORMALIZAÇÃO CONCLUÍDA!")
    print("="*60)
else:
    print("❌ Não foi possível normalizar: df_raw não existe")

🔧 NORMALIZANDO ESTRUTURA JSON

📊 ANTES da normalização: 6 colunas

🔄 Aplicando pd.json_normalize()...
   ✅ Normalização concluída!

📊 DEPOIS da normalização: 21 colunas

📋 Colunas normalizadas (com prefixos):
    1. customerID
    2. Churn
    3. customer.gender
    4. customer.SeniorCitizen
    5. customer.Partner
    6. customer.Dependents
    7. customer.tenure
    8. phone.PhoneService
    9. phone.MultipleLines
   10. internet.InternetService
   11. internet.OnlineSecurity
   12. internet.OnlineBackup
   13. internet.DeviceProtection
   14. internet.TechSupport
   15. internet.StreamingTV
   16. internet.StreamingMovies
   17. account.Contract
   18. account.PaperlessBilling
   19. account.PaymentMethod
   20. account.Charges.Monthly
   21. account.Charges.Total

🔍 Preview:


,customerID,Churn,customer.gender,customer.SeniorCitizen,customer.Partner,customer.Dependents,customer.tenure,phone.PhoneService,phone.MultipleLines,internet.InternetService,internet.OnlineSecurity,internet.OnlineBackup,internet.DeviceProtection,internet.TechSupport,internet.StreamingTV,internet.StreamingMovies,account.Contract,account.PaperlessBilling,account.PaymentMethod,account.Charges.Monthly,account.Charges.Total
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,No,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.60,593.3
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,No,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.90,542.4
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,No,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.90,280.85
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.00,1237.85
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,No,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.90,267.4



✅ NORMALIZAÇÃO CONCLUÍDA!


In [6]:
# ====================================================================
# 🌐 TRADUÇÃO DE COLUNAS PARA PORTUGUÊS
# ====================================================================
# Seguindo recomendação dos instrutores da Alura + preparação para ML
# ====================================================================

print("="*60)
print("🌐 TRADUZINDO COLUNAS PARA PORTUGUÊS")
print("="*60)

# Dicionário completo de tradução
traducao_colunas = {
    # Identificação
    'customerID': 'id_cliente',

    # Variável alvo (target para ML)
    'Churn': 'cancelou',

    # Demográficas
    'customer.gender': 'genero',
    'customer.SeniorCitizen': 'idoso',
    'customer.Partner': 'tem_parceiro',
    'customer.Dependents': 'tem_dependentes',
    'customer.tenure': 'meses_cliente',

    # Serviços telefônicos
    'phone.PhoneService': 'servico_telefone',
    'phone.MultipleLines': 'linhas_multiplas',

    # Serviços de internet
    'internet.InternetService': 'tipo_internet',
    'internet.OnlineSecurity': 'seguranca_online',
    'internet.OnlineBackup': 'backup_online',
    'internet.DeviceProtection': 'protecao_dispositivo',
    'internet.TechSupport': 'suporte_tecnico',
    'internet.StreamingTV': 'streaming_tv',
    'internet.StreamingMovies': 'streaming_filmes',

    # Informações de conta
    'account.Contract': 'tipo_contrato',
    'account.PaperlessBilling': 'fatura_digital',
    'account.PaymentMethod': 'metodo_pagamento',
    'account.Charges.Monthly': 'cobranca_mensal',
    'account.Charges.Total': 'cobranca_total'
}

print(f"\n📋 Traduzindo {len(traducao_colunas)} colunas...")

# Aplicar tradução
df_normalized.rename(columns=traducao_colunas, inplace=True)

print(f"✅ Colunas traduzidas com sucesso!")

print(f"\n📊 Colunas após tradução:")
for i, col in enumerate(df_normalized.columns, 1):
    tipo = df_normalized[col].dtype
    print(f"   {i:2d}. {col:25s} (tipo: {tipo})")

print("\n💡 Benefícios da tradução:")
print("   ✅ Código mais acessível para brasileiros")
print("   ✅ Facilita apresentações e relatórios")
print("   ✅ Preparado para ML em português (se necessário)")
print("   ✅ Segue recomendação dos instrutores Alura")

print("\n" + "="*60)
print("✅ EXTRAÇÃO COMPLETA - DADOS EM PORTUGUÊS!")
print("="*60)

🌐 TRADUZINDO COLUNAS PARA PORTUGUÊS

📋 Traduzindo 21 colunas...
✅ Colunas traduzidas com sucesso!

📊 Colunas após tradução:
    1. id_cliente                (tipo: object)
    2. cancelou                  (tipo: object)
    3. genero                    (tipo: object)
    4. idoso                     (tipo: int64)
    5. tem_parceiro              (tipo: object)
    6. tem_dependentes           (tipo: object)
    7. meses_cliente             (tipo: int64)
    8. servico_telefone          (tipo: object)
    9. linhas_multiplas          (tipo: object)
   10. tipo_internet             (tipo: object)
   11. seguranca_online          (tipo: object)
   12. backup_online             (tipo: object)
   13. protecao_dispositivo      (tipo: object)
   14. suporte_tecnico           (tipo: object)
   15. streaming_tv              (tipo: object)
   16. streaming_filmes          (tipo: object)
   17. tipo_contrato             (tipo: object)
   18. fatura_digital            (tipo: object)
   19. metodo_

---

## 🔧 4. TRANSFORMAÇÃO E LIMPEZA (Transform)

### 🎯 Objetivo

Garantir qualidade e integridade dos dados através de limpeza rigorosa.

### 🛠️ Técnicas

1. **Análise de qualidade** - Nulos, duplicados, tipos, strings vazias
2. **Remoção de duplicados**
3. **Tratamento de missing values**
4. **Conversão de tipos**
5. **Padronização categórica**
6. **Análise e tratamento de valores vazios na variável alvo** ⚠️
7. **Validação final**

---

In [8]:
# ====================================================================
# 🔍 ANÁLISE DE QUALIDADE DOS DADOS
# ====================================================================

print("="*60)
print("🔍 RELATÓRIO DE QUALIDADE DOS DADOS")
print("="*60)

# 1. VALORES NULOS
print("\n📊 1. VALORES NULOS")
print("-"*60)

nulos = df_normalized.isnull().sum()
if nulos.sum() == 0:
    print("✅ Nenhum valor nulo encontrado!")
else:
    print("⚠️ Valores nulos detectados:")
    for col, count in nulos[nulos > 0].items():
        pct = (count / len(df_normalized)) * 100
        print(f"   • {col}: {count:,} ({pct:.2f}%)")

# 2. DUPLICADOS
print("\n📊 2. DUPLICADOS")
print("-"*60)

duplicados = df_normalized.duplicated().sum()
if duplicados == 0:
    print("✅ Nenhum duplicado encontrado!")
else:
    print(f"⚠️ {duplicados:,} registros duplicados ({(duplicados/len(df_normalized)*100):.2f}%)")

# 3. TIPOS DE DADOS
print("\n📊 3. TIPOS DE DADOS")
print("-"*60)

tipos_esperados = {
    'id_cliente': 'object',
    'cancelou': 'object',
    'genero': 'object',
    'idoso': 'int64',
    'tem_parceiro': 'object',
    'tem_dependentes': 'object',
    'meses_cliente': 'int64',
    'servico_telefone': 'object',
    'linhas_multiplas': 'object',
    'tipo_internet': 'object',
    'seguranca_online': 'object',
    'backup_online': 'object',
    'protecao_dispositivo': 'object',
    'suporte_tecnico': 'object',
    'streaming_tv': 'object',
    'streaming_filmes': 'object',
    'tipo_contrato': 'object',
    'fatura_digital': 'object',
    'metodo_pagamento': 'object',
    'cobranca_mensal': 'float64',
    'cobranca_total': 'float64'
}

tipos_erro = []
for col, tipo_esperado in tipos_esperados.items():
    tipo_atual = str(df_normalized[col].dtype)
    status = "✅" if tipo_atual == tipo_esperado else "⚠️"
    if tipo_atual != tipo_esperado:
        tipos_erro.append((col, tipo_atual, tipo_esperado))
    print(f"{status} {col:25s} | Atual: {tipo_atual:10s} | Esperado: {tipo_esperado:10s}")

if tipos_erro:
    print(f"\n⚠️ {len(tipos_erro)} colunas precisam correção")
else:
    print(f"\n✅ Todos os tipos corretos!")

# 4. STRINGS VAZIAS ⭐ CRÍTICO
print("\n📊 4. STRINGS VAZIAS EM COLUNAS CATEGÓRICAS")
print("-"*60)

colunas_criticas = ['cancelou', 'genero', 'tipo_contrato', 'metodo_pagamento', 'tipo_internet']
strings_vazias_encontradas = False

for col in colunas_criticas:
    if col in df_normalized.columns:
        vazios = ((df_normalized[col] == '') | (df_normalized[col] == ' ')).sum()

        if vazios > 0:
            strings_vazias_encontradas = True
            pct = (vazios / len(df_normalized)) * 100
            print(f"⚠️ {col}: {vazios} strings vazias ({pct:.2f}%)")

if not strings_vazias_encontradas:
    print("✅ Nenhuma string vazia encontrada!")

print("\n" + "="*60)
print("📊 ANÁLISE DE QUALIDADE CONCLUÍDA")
print("="*60)

🔍 RELATÓRIO DE QUALIDADE DOS DADOS

📊 1. VALORES NULOS
------------------------------------------------------------
✅ Nenhum valor nulo encontrado!

📊 2. DUPLICADOS
------------------------------------------------------------
✅ Nenhum duplicado encontrado!

📊 3. TIPOS DE DADOS
------------------------------------------------------------
✅ id_cliente                | Atual: object     | Esperado: object    
✅ cancelou                  | Atual: object     | Esperado: object    
✅ genero                    | Atual: object     | Esperado: object    
✅ idoso                     | Atual: int64      | Esperado: int64     
✅ tem_parceiro              | Atual: object     | Esperado: object    
✅ tem_dependentes           | Atual: object     | Esperado: object    
✅ meses_cliente             | Atual: int64      | Esperado: int64     
✅ servico_telefone          | Atual: object     | Esperado: object    
✅ linhas_multiplas          | Atual: object     | Esperado: object    
✅ tipo_internet       

In [9]:
# ====================================================================
# 📋 CRIAR CÓPIA PARA LIMPEZA
# ====================================================================

print("📋 Criando cópia para limpeza...")
df_clean = df_normalized.copy()
print(f"✅ df_clean criado: {df_clean.shape[0]:,} × {df_clean.shape[1]}")

# ====================================================================
# 🗑️ REMOVER DUPLICADOS
# ====================================================================

print("\n" + "="*60)
print("🗑️ REMOVENDO DUPLICADOS")
print("="*60)

duplicados_antes = df_clean.duplicated().sum()
print(f"\n📊 Duplicados ANTES: {duplicados_antes}")

if duplicados_antes > 0:
    df_clean.drop_duplicates(keep='first', inplace=True)
    df_clean.reset_index(drop=True, inplace=True)
    print(f"✅ {duplicados_antes} removidos")
else:
    print("✅ Nenhum duplicado para remover!")

print(f"\n📊 Dataset: {df_clean.shape[0]:,} registros × {df_clean.shape[1]} colunas")
print("="*60)

📋 Criando cópia para limpeza...
✅ df_clean criado: 7,267 × 21

🗑️ REMOVENDO DUPLICADOS

📊 Duplicados ANTES: 0
✅ Nenhum duplicado para remover!

📊 Dataset: 7,267 registros × 21 colunas


In [10]:
# ====================================================================
# 🔧 TRATAMENTO DA COLUNA cobranca_total
# ====================================================================

print("="*60)
print("🔧 TRATANDO cobranca_total")
print("="*60)

print(f"\n📊 Tipo atual: {df_clean['cobranca_total'].dtype}")

# Verificar valores vazios
valores_vazios = df_clean['cobranca_total'] == ' '
total_vazios = valores_vazios.sum()

print(f"\n⚠️ Valores vazios (' '): {total_vazios}")

if total_vazios > 0:
    print(f"\n🔍 Perfil desses clientes:")
    exemplos = df_clean[valores_vazios][['id_cliente', 'meses_cliente', 'cobranca_mensal', 'cobranca_total']].head(5)
    display(exemplos)

    print(f"\n💡 Clientes com cobranca_total vazia geralmente têm meses_cliente baixo (novos)")
    print(f"   ➡️ Substituir por 0.0 (nenhuma cobrança acumulada ainda)")

# Substituir e converter
df_clean['cobranca_total'] = df_clean['cobranca_total'].replace(' ', np.nan)
df_clean['cobranca_total'] = pd.to_numeric(df_clean['cobranca_total'], errors='coerce')
df_clean['cobranca_total'].fillna(0.0, inplace=True)

print(f"\n✅ Tipo após conversão: {df_clean['cobranca_total'].dtype}")
print(f"✅ Valores NaN restantes: {df_clean['cobranca_total'].isna().sum()}")

print("\n" + "="*60)

🔧 TRATANDO cobranca_total

📊 Tipo atual: object

⚠️ Valores vazios (' '): 11

🔍 Perfil desses clientes:


,id_cliente,meses_cliente,cobranca_mensal,cobranca_total
975,1371-DWPAZ,0,56.05,
1775,2520-SGTTA,0,20.00,
1955,2775-SEFEE,0,61.90,
2075,2923-ARZLG,0,19.70,
2232,3115-CZMZD,0,20.25,



💡 Clientes com cobranca_total vazia geralmente têm meses_cliente baixo (novos)
   ➡️ Substituir por 0.0 (nenhuma cobrança acumulada ainda)

✅ Tipo após conversão: float64
✅ Valores NaN restantes: 0



In [11]:
# ====================================================================
# 🔄 CONVERSÃO DE TIPOS DE DADOS
# ====================================================================

print("="*60)
print("🔄 GARANTINDO TIPOS CORRETOS")
print("="*60)

# Garantir tipos numéricos
df_clean['idoso'] = df_clean['idoso'].astype('int64')
df_clean['meses_cliente'] = df_clean['meses_cliente'].astype('int64')
df_clean['cobranca_mensal'] = df_clean['cobranca_mensal'].astype('float64')
df_clean['cobranca_total'] = df_clean['cobranca_total'].astype('float64')

print("\n✅ Tipos convertidos:")
print(f"   • idoso: {df_clean['idoso'].dtype}")
print(f"   • meses_cliente: {df_clean['meses_cliente'].dtype}")
print(f"   • cobranca_mensal: {df_clean['cobranca_mensal'].dtype}")
print(f"   • cobranca_total: {df_clean['cobranca_total'].dtype}")

print(f"\n📊 Valores nulos totais: {df_clean.isnull().sum().sum()}")
print("="*60)

🔄 GARANTINDO TIPOS CORRETOS

✅ Tipos convertidos:
   • idoso: int64
   • meses_cliente: int64
   • cobranca_mensal: float64
   • cobranca_total: float64

📊 Valores nulos totais: 0


In [12]:
# ====================================================================
# 🔬 ANÁLISE PROFUNDA: VALORES VAZIOS EM 'cancelou' (VARIÁVEL ALVO)
# ====================================================================
# ⚠️ DECISÃO CRÍTICA: O que fazer com registros sem valor em 'cancelou'?
# ====================================================================

print("="*60)
print("🔬 ANÁLISE PROFUNDA: VALORES VAZIOS EM 'cancelou'")
print("="*60)

# Identificar valores vazios
mask_vazios = (df_clean['cancelou'] == '') | (df_clean['cancelou'] == ' ') | (df_clean['cancelou'].isna())
total_vazios = mask_vazios.sum()

print(f"\n📊 SITUAÇÃO ATUAL:")
print(f"   • Total de registros: {len(df_clean):,}")
print(f"   • Registros com 'cancelou' vazio: {total_vazios}")
print(f"   • Percentual: {(total_vazios/len(df_clean)*100):.2f}%")

if total_vazios > 0:

    print(f"\n" + "="*60)
    print(f"🔍 INVESTIGANDO OS {total_vazios} REGISTROS PROBLEMÁTICOS")
    print("="*60)

    # Extrair registros problemáticos
    df_vazios = df_clean[mask_vazios].copy()

    print(f"\n📋 Amostra (primeiros 10):")
    colunas_analise = ['id_cliente', 'cancelou', 'meses_cliente', 'cobranca_mensal',
                       'cobranca_total', 'tipo_contrato', 'tipo_internet']
    display(df_vazios[colunas_analise].head(10))

    # Análise estatística comparativa
    print(f"\n📊 COMPARAÇÃO ESTATÍSTICA:")
    print(f"\n   {'Variável':25s} | {'Com Vazio':>15s} | {'Sem Vazio':>15s} | {'Diferença':>12s}")
    print("   " + "-"*75)

    df_sem_vazios = df_clean[~mask_vazios]

    # Variáveis numéricas
    for col in ['meses_cliente', 'cobranca_mensal', 'cobranca_total']:
        media_vazios = df_vazios[col].mean()
        media_sem_vazios = df_sem_vazios[col].mean()
        dif = abs(media_vazios - media_sem_vazios)
        print(f"   {col:25s} | {media_vazios:>15.2f} | {media_sem_vazios:>15.2f} | {dif:>12.2f}")

    # Variáveis categóricas (distribuição)
    print(f"\n   Distribuição categórica:")
    for col in ['tipo_contrato', 'tipo_internet']:
        print(f"\n   {col}:")
        dist_vazios = df_vazios[col].value_counts(normalize=True).head(3)
        for val, pct in dist_vazios.items():
            print(f"      {val}: {pct*100:.1f}%")

    # ====================================================================
    # 📋 DECISÃO TÉCNICA: POR QUE REMOVER?
    # ====================================================================

    print(f"\n" + "="*60)
    print(f"⚖️ DECISÃO TÉCNICA: REMOVER OS {total_vazios} REGISTROS")
    print("="*60)

    print(f"\n📌 JUSTIFICATIVA:")
    print(f"""
   1. 🎯 'cancelou' é a VARIÁVEL ALVO (target para ML)
      • Não podemos prever o que não sabemos
      • Imputar a variável alvo = inventar dados = ERRO GRAVE

   2. 📊 Impacto Estatístico Aceitável
      • Apenas {(total_vazios/len(df_clean)*100):.2f}% dos dados
      • Não há viés demográfico aparente (distribuição similar)

   3. 🔬 Alternativas Avaliadas e Rejeitadas:
      ❌ Imputar com moda (Yes/No) → Vicia o modelo
      ❌ Criar categoria "Desconhecido" → Incompatível com ML binário
      ❌ Imputar com modelo preditivo → Lógica circular

   4. ✅ Decisão Final: REMOVER
      • É a ÚNICA opção tecnicamente correta
      • Mantém integridade dos dados
      • Segue boas práticas de Data Science
    """)

    print(f"\n📚 Referências:")
    print(f"   • 'Hands-On Machine Learning' (Aurélien Géron)")
    print(f"   • 'Data Science for Business' (Provost & Fawcett)")
    print(f"   • Consenso da comunidade: NUNCA imputar variável target")

    # ====================================================================
    # 🗑️ EXECUTAR REMOÇÃO
    # ====================================================================

    print(f"\n" + "="*60)
    print(f"🗑️ EXECUTANDO REMOÇÃO")
    print("="*60)

    registros_antes = len(df_clean)

    # Remover registros
    df_clean = df_clean[~mask_vazios].copy()
    df_clean.reset_index(drop=True, inplace=True)

    registros_depois = len(df_clean)
    removidos = registros_antes - registros_depois

    print(f"\n✅ Remoção concluída:")
    print(f"   • Antes: {registros_antes:,} registros")
    print(f"   • Depois: {registros_depois:,} registros")
    print(f"   • Removidos: {removidos} ({(removidos/registros_antes*100):.2f}%)")

    # Verificar valores restantes em 'cancelou'
    print(f"\n✅ Valores únicos em 'cancelou' após limpeza:")
    print(df_clean['cancelou'].value_counts())

    print(f"\n💡 RESULTADO: Dataset limpo e pronto para análise/ML!")

else:
    print("\n✅ Não há valores vazios em 'cancelou'!")

print("\n" + "="*60)

🔬 ANÁLISE PROFUNDA: VALORES VAZIOS EM 'cancelou'

📊 SITUAÇÃO ATUAL:
   • Total de registros: 7,267
   • Registros com 'cancelou' vazio: 224
   • Percentual: 3.08%

🔍 INVESTIGANDO OS 224 REGISTROS PROBLEMÁTICOS

📋 Amostra (primeiros 10):


,id_cliente,cancelou,meses_cliente,cobranca_mensal,cobranca_total,tipo_contrato,tipo_internet
30,0047-ZHDTW,,11,79.00,929.30,Month-to-month,Fiber optic
75,0120-YZLQA,,71,19.90,1355.10,Two year,No
96,0154-QYHJU,,29,58.75,1696.20,One year,DSL
98,0162-RZGMZ,,5,59.90,287.85,Month-to-month,DSL
175,0274-VVQOQ,,65,103.15,6792.45,One year,Fiber optic
219,0328-IBUPK,,51,34.20,1782.00,Two year,DSL
312,0448-YZNZE,,27,95.55,2510.20,Month-to-month,Fiber optic
351,0510-EXSMQ,,9,69.05,651.50,Month-to-month,Fiber optic
368,0530-HBKHZ,,1,24.80,24.80,Month-to-month,DSL
374,0534-JRNIG,,6,93.55,536.40,Month-to-month,Fiber optic



📊 COMPARAÇÃO ESTATÍSTICA:

   Variável                  |       Com Vazio |       Sem Vazio |    Diferença
   ---------------------------------------------------------------------------
   meses_cliente             |           31.57 |           32.37 |         0.80
   cobranca_mensal           |           63.41 |           64.76 |         1.35
   cobranca_total            |         2196.93 |         2279.73 |        82.80

   Distribuição categórica:

   tipo_contrato:
      Month-to-month: 58.0%
      Two year: 21.4%
      One year: 20.5%

   tipo_internet:
      Fiber optic: 45.5%
      DSL: 29.9%
      No: 24.6%

⚖️ DECISÃO TÉCNICA: REMOVER OS 224 REGISTROS

📌 JUSTIFICATIVA:

   1. 🎯 'cancelou' é a VARIÁVEL ALVO (target para ML)
      • Não podemos prever o que não sabemos
      • Imputar a variável alvo = inventar dados = ERRO GRAVE
   
   2. 📊 Impacto Estatístico Aceitável
      • Apenas 3.08% dos dados
      • Não há viés demográfico aparente (distribuição similar)
   
   3. 🔬 A

In [13]:
# ====================================================================
# 📝 PADRONIZAÇÃO DE VALORES CATEGÓRICOS
# ====================================================================

print("="*60)
print("📝 PADRONIZANDO VALORES CATEGÓRICOS")
print("="*60)

# Aplicar strip em todas colunas categóricas
colunas_categoricas = df_clean.select_dtypes(include='object').columns

print(f"\n🔄 Aplicando strip() em {len(colunas_categoricas)} colunas...")

for col in colunas_categoricas:
    df_clean[col] = df_clean[col].str.strip()

print("   ✅ Espaços extras removidos")

# Verificar valores únicos principais
print(f"\n📊 Valores únicos após padronização:")
colunas_verificar = ['cancelou', 'genero', 'tipo_internet', 'tipo_contrato']

for col in colunas_verificar:
    if col in df_clean.columns:
        print(f"\n   {col}: {df_clean[col].unique().tolist()}")

print("\n" + "="*60)
print("✅ PADRONIZAÇÃO CONCLUÍDA")
print("="*60)

📝 PADRONIZANDO VALORES CATEGÓRICOS

🔄 Aplicando strip() em 17 colunas...
   ✅ Espaços extras removidos

📊 Valores únicos após padronização:

   cancelou: ['No', 'Yes']

   genero: ['Female', 'Male']

   tipo_internet: ['DSL', 'Fiber optic', 'No']

   tipo_contrato: ['One year', 'Month-to-month', 'Two year']

✅ PADRONIZAÇÃO CONCLUÍDA


In [14]:
# ====================================================================
# ✅ VALIDAÇÃO FINAL DA TRANSFORMAÇÃO
# ====================================================================

print("="*60)
print("✅ VALIDAÇÃO FINAL")
print("="*60)

# Checklist
print(f"\n📋 CHECKLIST DE QUALIDADE:")
print("-"*60)

# 1. Colunas
total_colunas = df_clean.shape[1]
check = "✅" if total_colunas == 21 else "❌"
print(f"{check} Colunas: {total_colunas} (esperado: 21)")

# 2. Nulos
total_nulos = df_clean.isnull().sum().sum()
check = "✅" if total_nulos == 0 else "⚠️"
print(f"{check} Valores nulos: {total_nulos} (esperado: 0)")

# 3. Duplicados
total_duplicados = df_clean.duplicated().sum()
check = "✅" if total_duplicados == 0 else "❌"
print(f"{check} Duplicados: {total_duplicados} (esperado: 0)")

# 4. Tipos
print(f"\n📊 Validação de tipos:")
tipos_ok = True
for col in ['idoso', 'meses_cliente']:
    if df_clean[col].dtype != 'int64':
        tipos_ok = False
for col in ['cobranca_mensal', 'cobranca_total']:
    if df_clean[col].dtype != 'float64':
        tipos_ok = False

check = "✅" if tipos_ok else "❌"
print(f"{check} Tipos de dados validados")

# 5. Variável alvo
valores_cancelou = df_clean['cancelou'].unique()
check = "✅" if len(valores_cancelou) == 2 and all(x in ['Yes', 'No'] for x in valores_cancelou) else "❌"
print(f"{check} Variável 'cancelou': {valores_cancelou.tolist()}")

# Resumo
print("\n" + "="*60)
print("📊 RESUMO DO DATASET LIMPO:")
print(f"   • Registros: {df_clean.shape[0]:,}")
print(f"   • Variáveis: {df_clean.shape[1]}")
print(f"   • Memória: {df_clean.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print(f"   • Nulos: {df_clean.isnull().sum().sum()}")
print(f"   • Duplicados: {df_clean.duplicated().sum()}")

print(f"\n🔍 Preview (5 linhas):")
display(df_clean.head())

print("\n" + "="*60)
print("✅ TRANSFORMAÇÃO CONCLUÍDA COM SUCESSO!")
print("="*60)

✅ VALIDAÇÃO FINAL

📋 CHECKLIST DE QUALIDADE:
------------------------------------------------------------
✅ Colunas: 21 (esperado: 21)
✅ Valores nulos: 0 (esperado: 0)
✅ Duplicados: 0 (esperado: 0)

📊 Validação de tipos:
✅ Tipos de dados validados
✅ Variável 'cancelou': ['No', 'Yes']

📊 RESUMO DO DATASET LIMPO:
   • Registros: 7,043
   • Variáveis: 21
   • Memória: 6.51 MB
   • Nulos: 0
   • Duplicados: 0

🔍 Preview (5 linhas):


,id_cliente,cancelou,genero,idoso,tem_parceiro,tem_dependentes,meses_cliente,servico_telefone,linhas_multiplas,tipo_internet,seguranca_online,backup_online,protecao_dispositivo,suporte_tecnico,streaming_tv,streaming_filmes,tipo_contrato,fatura_digital,metodo_pagamento,cobranca_mensal,cobranca_total
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,No,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.60,593.30
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,No,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.90,542.40
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,No,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.90,280.85
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.00,1237.85
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,No,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.90,267.40



✅ TRANSFORMAÇÃO CONCLUÍDA COM SUCESSO!


In [15]:
# ====================================================================
# 💾 CARGA (LOAD) - SALVAR DATASET LIMPO
# ====================================================================

print("="*60)
print("💾 SALVANDO DATASET LIMPO")
print("="*60)

# Nome do arquivo em português
arquivo_csv = "telecom_limpo.csv"

print(f"\n🔄 Salvando '{arquivo_csv}'...")
df_clean.to_csv(arquivo_csv, index=False, encoding='utf-8')

# Verificar arquivo
import os
if os.path.exists(arquivo_csv):
    tamanho_mb = os.path.getsize(arquivo_csv) / (1024 * 1024)
    print(f"✅ Arquivo salvo com sucesso!")
    print(f"\n📄 Detalhes:")
    print(f"   • Nome: {arquivo_csv}")
    print(f"   • Registros: {len(df_clean):,}")
    print(f"   • Colunas: {len(df_clean.columns)}")
    print(f"   • Tamanho: {tamanho_mb:.2f} MB")
    print(f"   • Localização: {os.path.abspath(arquivo_csv)}")

# Download automático
print(f"\n📥 Iniciando download automático...")
try:
    from google.colab import files
    files.download(arquivo_csv)
    print(f"✅ Download iniciado! Verifique sua pasta Downloads")
except:
    print(f"⚠️ Não está no Colab - download manual necessário")

# ====================================================================
# 🎉 RESUMO COMPLETO DO ETL
# ====================================================================

print("\n" + "="*60)
print("🎉 PROCESSO ETL CONCLUÍDO COM SUCESSO!")
print("="*60)

print(f"""
📊 RESUMO DO ETL:

   ✅ EXTRACT (Extração):
      • Fonte: API REST (JSON)
      • Registros extraídos: {df_raw.shape[0]:,}
      • Estrutura: 6 colunas aninhadas

   ✅ TRANSFORM (Transformação):
      • JSON normalizado → 21 colunas planas
      • Colunas traduzidas para português
      • 224 registros com 'cancelou' vazio removidos (3.08%)
      • 0 duplicados
      • Tipos corrigidos
      • Categorias padronizadas

   ✅ LOAD (Carga):
      • Arquivo: {arquivo_csv}
      • Registros finais: {len(df_clean):,}
      • Colunas: {len(df_clean.columns)} (todas em português)
      • Tamanho: {tamanho_mb:.2f} MB

📦 DataFrames disponíveis:
   • df_raw: dados brutos (7,267 × 6)
   • df_normalized: normalizados (7,267 × 21)
   • df_clean: LIMPO E FINAL (7,043 × 21) ⭐

🎯 DATASET PRONTO PARA:
   ✅ Análise Exploratória de Dados (EDA)
   ✅ Visualizações e dashboards
   ✅ Machine Learning (tipos corretos, sem missing)
   ✅ Apresentações em português
""")

print("="*60)
print("📌 PRÓXIMO PASSO: ANÁLISE EXPLORATÓRIA (EDA)")
print("="*60)

print("\n⚠️ LEMBRE-SE:")
print("   • Salve o notebook (File → Download → .ipynb)")
print("   • Faça commit no GitHub do CSV + notebook")
print("   • O CSV foi baixado - guarde em local seguro!")

💾 SALVANDO DATASET LIMPO

🔄 Salvando 'telecom_limpo.csv'...
✅ Arquivo salvo com sucesso!

📄 Detalhes:
   • Nome: telecom_limpo.csv
   • Registros: 7,043
   • Colunas: 21
   • Tamanho: 0.93 MB
   • Localização: /content/telecom_limpo.csv

📥 Iniciando download automático...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Download iniciado! Verifique sua pasta Downloads

🎉 PROCESSO ETL CONCLUÍDO COM SUCESSO!

📊 RESUMO DO ETL:

   ✅ EXTRACT (Extração):
      • Fonte: API REST (JSON)
      • Registros extraídos: 7,267
      • Estrutura: 6 colunas aninhadas
   
   ✅ TRANSFORM (Transformação):
      • JSON normalizado → 21 colunas planas
      • Colunas traduzidas para português
      • 224 registros com 'cancelou' vazio removidos (3.08%)
      • 0 duplicados
      • Tipos corrigidos
      • Categorias padronizadas
   
   ✅ LOAD (Carga):
      • Arquivo: telecom_limpo.csv
      • Registros finais: 7,043
      • Colunas: 21 (todas em português)
      • Tamanho: 0.93 MB

📦 DataFrames disponíveis:
   • df_raw: dados brutos (7,267 × 6)
   • df_normalized: normalizados (7,267 × 21)
   • df_clean: LIMPO E FINAL (7,043 × 21) ⭐

🎯 DATASET PRONTO PARA:
   ✅ Análise Exploratória de Dados (EDA)
   ✅ Visualizações e dashboards
   ✅ Machine Learning (tipos corretos, sem missing)
   ✅ Apresentações em português

📌 PRÓX